In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')[]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
print(device)

cuda


In [ ]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [ ]:
train_path='/content/drive/MyDrive/Scene_detection/seg_train/seg_train'
test_path='/content/drive/MyDrive/Scene_detection/seg_test/seg_test'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [ ]:
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [ ]:
class ConvNet(nn.Module):
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)

        output=self.pool(output)

        output=self.conv2(output)
        output=self.relu2(output)

        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)


            #Above output will be in matrix form, with shape (256,32,75,75)

        output=output.view(-1,32*75*75)


        output=self.fc(output)

        return output
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()

        #Output size after convolution filter
        #((w-f+2P)/s) +1

        #Input shape= (256,3,150,150)

        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)

        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)


        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)


        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)



In [ ]:
model=ConvNet(num_classes=6).to(device)

In [ ]:
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [ ]:
num_epochs=10

In [ ]:
train_count=len(glob.glob(train_path+'/*/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [ ]:
print(train_count,test_count)

14126 3000


In [ ]:
best_accuracy=0.0

for epoch in range(num_epochs):

    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    train_loss_graph=[]
    train_accuracy_graph=[]

    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())

        optimizer.zero_grad()

        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()



        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)

        train_accuracy+=int(torch.sum(prediction==labels.data))

    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    train_loss_graph.append(train_loss)
    train_accuracy_graph.append(train_accuracy)




In [ ]:
# Evaluation on testing dataset
for epoch in range(num_epochs):

    model.eval()

    test_accuracy=0.0
    test_accuracy_graph=[]

    test_loss_graph=[]
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())

        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))

    test_accuracy=test_accuracy/test_count
    test_accuracy_graph.append(test_accuracy)
    test_loss_graph.append(1-test_accuracy)

    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))

    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy

In [ ]:
import matplotlib.pyplot as plt
train_loss_graph.append(0.1732)
train_loss_graph.append(0.1701)
train_loss_graph.append(0.1756)
train_loss_graph.append(0.1715)
train_loss_graph.append(0.1699)
train_loss_graph.append(0.1626)
train_loss_graph.append(0.1786)
train_loss_graph.append(0.1666)
train_loss_graph.append(0.1702)
train_loss_graph.append(0.1732)


plt.plot(train_loss_graph)
plt.show()

In [ ]:
train_accuracy_graph.append( 0.9549766388220303)
train_accuracy_graph.append( 0.9549766388220303)
train_accuracy_graph.append( 0.9549766388220303)
train_accuracy_graph.append( 0.9549766388220303)
train_accuracy_graph.append( 0.9549766388220303)
train_accuracy_graph.append( 0.9549766388220303)
train_accuracy_graph.append( 0.9549766388220303)
train_accuracy_graph.append( 0.9549766388220303)
train_accuracy_graph.append( 0.9549766388220303)
train_accuracy_graph.append( 0.9549766388220303)
plt.plot(train_accuracy_graph)
plt.show()

In [ ]:
test_accuracy_graph.append(0.7533333333333333)
test_accuracy_graph.append(0.7566666666666667)
test_accuracy_graph.append(0.756)
test_accuracy_graph.append(0.752)
test_accuracy_graph.append(0.7513333333333333)
test_accuracy_graph.append(0.745)
test_accuracy_graph.append(0.7586666666666667)
test_accuracy_graph.append(0.753)
test_accuracy_graph.append(0.7516666666666667)
test_accuracy_graph.append(0.7496666666666667)

plt.plot(test_accuracy_graph)
plt.show()

In [ ]:
plt.plot(test_loss_graph)
plt.show()